In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import json
import csv
import time

def getLinks(url):
    html_page = urlopen(url)
    soup = BeautifulSoup(html_page)
    links = []

    for link in soup.findAll('a', attrs={'href': re.compile("/product-category/")}):
        links.append(link.get('href'))

    return links

a=getLinks("https://www.alraya.com.sa/?lang=ar") 
a

['https://www.alraya.com.sa/product-category/grocery-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/meat/?lang=ar',
 'https://www.alraya.com.sa/product-category/frozen-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/cosmetics/?lang=ar',
 'https://www.alraya.com.sa/product-category/milk-cream-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/grocery-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/meat/?lang=ar',
 'https://www.alraya.com.sa/product-category/frozen-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/cosmetics/?lang=ar',
 'https://www.alraya.com.sa/product-category/milk-cream-ar/?lang=ar']

In [5]:
link=list(set(a))
link

['https://www.alraya.com.sa/product-category/frozen-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/meat/?lang=ar',
 'https://www.alraya.com.sa/product-category/grocery-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/milk-cream-ar/?lang=ar',
 'https://www.alraya.com.sa/product-category/cosmetics/?lang=ar']

In [15]:
global writer
global filecsv
csv_columns = ['name','price']
filecsv = open('product.csv', 'w',encoding='utf8')
writer = csv.DictWriter(filecsv, fieldnames=csv_columns)
writer.writeheader()
for i in link:
    url=i
    url=url.replace("?lang=ar","")
    file = open('product.json','w',encoding='utf8')
    file.write('[\n')
    data = {}
    
    for page in range(1,30):
        print('---', page, '---')
        r = requests.get(url + "page/"+str(page)+"/"+"?lang=ar")
        print(url + "page/"+str(page)+"/"+"?lang=ar")
        soup = BeautifulSoup(r.text, "html.parser")
        ancher=soup.find_all('div',{'class' : 'product-block grid'})
        
        for pt in  ancher:
            name=pt.find_all("a",{'class' : 'product-image'})[0].get("title")
            itemPrice=pt.find_all('span', {'class' : 'woocommerce-Price-amount amount'})
            try:
                price_result=re.search("</span>.*</span>",str(itemPrice[0]))
                price_result=price_result.group(0)
                price_result=price_result.replace("</span>"," ")
            except:
                print("not priced")
                price_result=-1
            print(pt)
            print(name)
            print(itemPrice)
            writer.writerow({'name': name, 'price': price_result})
            data['name'] =name
            data['price'] =price_result
            json_data = json.dumps(data,ensure_ascii=False)
            file.write(json_data)
            file.write(",\n")  
                       
file.write("\n]")
filecsv.close()
file.close()

--- 1 ---
https://www.alraya.com.sa/product-category/frozen-ar/page/1/?lang=ar
<div class="product-block grid" data-product-id="4611"><div class="product-content"><div class="block-inner"><figure class="image"> <a class="product-image" href="https://www.alraya.com.sa/product/%d8%a3%d9%83%d8%a8%d8%a7%d8%af-%d8%af%d8%ac%d8%a7%d8%ac/?lang=ar" title="أكباد دجاج"> <img alt="" class="attachment-woocommerce_thumbnail size-woocommerce_thumbnail" data-lazy-sizes="(max-width: 405px) 100vw, 405px" data-lazy-src="https://www.alraya.com.sa/wp-content/uploads/2019/09/اكباد-دجاج-مجمده-ساديا-01-405x330.png" data-lazy-srcset="https://www.alraya.com.sa/wp-content/uploads/2019/09/اكباد-دجاج-مجمده-ساديا-01-405x330.png 405w, https://www.alraya.com.sa/wp-content/uploads/2019/09/اكباد-دجاج-مجمده-ساديا-01-160x130.png 160w" height="330" src="data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20405%20330'%3E%3C/svg%3E" width="405"><noscript><img alt="" class="attachment-woocommerce_

In [19]:
import pandas as pd 
import numpy as np
df = pd.read_csv('product.csv')
df.head(20)

,name,price
0,أكباد دجاج,15.75
1,أنصاف صدور دجاج مجمدة,16.80
2,أنصاف صدور دجاج مجمدة,49.49
3,المنتج 1,20.00
4,دجاج كامل ساديا,13.76
5,دجاج كامل سيارا,11.78
6,دجاج مجمد,16.50
7,زنجز فيليه الدجاج أميركانا,46.95
8,زنجز فيليه الدجاج أميركانا,46.95
9,سادية,-1.00


In [21]:
df.to_excel("product.xlsx")